In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import math

import pandas as pd
import numpy as np
import pennylane as qml
from pennylane import numpy as qnp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

First we load the datasets:

In [2]:
gluehwein = 'gluehweindorf'
krampus = 'krampuskogel'
lebkuchen = 'lebkuchenstadt'

villages = [gluehwein, krampus, lebkuchen]
datasets = {}

for village in villages:
        datasets[village] = pd.read_csv(f'{village}.csv')

There are no null values in the dataset and 500 entries

In [3]:
datasets[gluehwein].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   carol_singing    500 non-null    float64
 1   snowball_energy  500 non-null    float64
 2   label            500 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


In [4]:
datasets[krampus].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   carol_singing    500 non-null    float64
 1   snowball_energy  500 non-null    float64
 2   label            500 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


In [5]:
datasets[lebkuchen].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   carol_singing    500 non-null    float64
 1   snowball_energy  500 non-null    float64
 2   label            500 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


Min-Max values

In [6]:
for name in (gluehwein, krampus, lebkuchen):
    print(f'{name}-carol_singing: min: {datasets[name]['carol_singing'].min()}, max: {datasets[name]['carol_singing'].max()}')
    print(f'{name}-snowball_energy: min: {datasets[name]['snowball_energy'].min()}, max: {datasets[name]['snowball_energy'].max()}')
    print()

gluehweindorf-carol_singing: min: -2.9804047381428838, max: 2.9371221242497527
gluehweindorf-snowball_energy: min: -2.9723258785728337, max: 3.141592653589793

krampuskogel-carol_singing: min: -3.141592653589793, max: 3.117552213512825
krampuskogel-snowball_energy: min: -2.8957902158960995, max: 2.8898416183940667

lebkuchenstadt-carol_singing: min: -1.620977694446723, max: 3.141592653589793
lebkuchenstadt-snowball_energy: min: -0.6755685974503365, max: 1.4214525718679278



Label values

In [7]:
for name in (gluehwein, krampus, lebkuchen):
    print(f'{name}-labels:  {datasets[gluehwein]['label'].unique()}')


gluehweindorf-labels:  [1 0]
krampuskogel-labels:  [1 0]
lebkuchenstadt-labels:  [1 0]


In [8]:
X_train = {}
y_train = {}

for name in (gluehwein, krampus, lebkuchen):
    X_train[name] = datasets[name][['carol_singing', 'snowball_energy']].values
    y_train[name] = datasets[name]['label'].values

# Encoding

## Amplitude encoding

In [9]:
def amplitude_encoding(inputs, wires):
    qml.AmplitudeEmbedding(
        features=inputs,
        wires=wires,
        normalize=True,
        pad_with=0.0
    )


## Angle encoding

In [10]:
def angle_encoding(inputs, wires):
    qml.AngleEmbedding(
        features=inputs, 
        wires=wires, 
        rotation='Y')

## Density matrix encoding


In [11]:
def density_matrix_encoding(inputs, wires):    
    x0, x1 = inputs
    rho = np.outer(inputs, inputs) / np.inner(inputs, inputs)

    qml.QubitDensityMatrix(rho, wires=wires)

# Entanglement

## Linear Entanglement

In [12]:
def linear_entanglement(wires):
    for i in range(len(wires) - 1):
        qml.CNOT(wires=[wires[i], wires[i + 1]])

## Circular Entanglement

In [13]:
def circular_entanglement(wires):
    linear_entanglement(wires)
    qml.CNOT(wires=[wires[-1], wires[0]])


## Full Entanglement

In [14]:
def full_entanglement(wires):
    for i in range(len(wires)):
        for j in range(i + 1, len(wires)):
            qml.CNOT(wires=[wires[i], wires[j]])


# QML Circuit

In [15]:
def make_qml_circuit(dev):
    
    @qml.qnode(dev)
    def qml_circuit(weights, inputs, encoding_fn, entanglement_fn, reuploading_count):
        encoding_fn(inputs, wires=list(range(num_qubits)))

        for j in range(reuploading_count):
            entanglement_fn(wires=list(range(num_qubits)))
            for i in range(num_qubits):
                qml.RY(weights[j][i], wires=i)

        return qml.expval(qml.PauliZ(0)) # this returns a number between the eigenvalues of Z, which are -1 and 1
    
    return qml_circuit

# Cost

In [16]:
def cost(weights, param):
    [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count] = param
    predictions = qnp.array([
        (qml_circuit(weights, x, encoding_fn, entanglement_fn, reuploading_count) * 0.5 + 0.5)
        for x in X
    ])
    eps = 1e-7
    
    return -qnp.mean(
        Y * qnp.log(predictions + eps) +
        (1 - Y) * qnp.log(1 - predictions + eps)
    )


In [17]:
def predict_proba(weights, X, qml_circuit, encoding_fn, entanglement_fn, reuploading_count):
    return qnp.array([
        (qml_circuit(weights, x, encoding_fn, entanglement_fn, reuploading_count) + 1) / 2
        for x in X
    ])


def predict(weights, X, qml_circuit, encoding_fn, entanglement_fn, reuploading_count):
    probs = predict_proba(
        weights, X, qml_circuit, encoding_fn, entanglement_fn, reuploading_count
    )
    return (probs >= 0.5).astype(int)


def accuracy(y_true, y_pred):
    return qnp.mean(y_true == y_pred)


def confusion_matrix(y_true, y_pred):
    tp = qnp.sum((y_true == 1) & (y_pred == 1))
    tn = qnp.sum((y_true == 0) & (y_pred == 0))
    fp = qnp.sum((y_true == 0) & (y_pred == 1))
    fn = qnp.sum((y_true == 1) & (y_pred == 0))

    return qnp.array([[tn, fp],
                      [fn, tp]])


# Evaluation

## Gluehweindorf

In [18]:
opt = qml.AdamOptimizer(stepsize=0.01)

num_qubits = 2
dev = qml.device("default.qubit", wires=num_qubits)
qml_circuit = make_qml_circuit(dev)

encoding_fn = amplitude_encoding
entanglement_fn = linear_entanglement 

X = X_train[gluehwein]
Y = y_train[gluehwein]


for reuploading_count in (1, 3):
    print(f"\n=== Reuploading count: {reuploading_count} ===")

    weights = qnp.array(
        np.random.uniform(0, math.pi, size=(reuploading_count, num_qubits)),
        requires_grad=True
    )

    param = [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count]

    for epoch in range(100):
        args, current_cost = opt.step_and_cost(cost, weights, param)
        weights = args[0]

        # ---- evaluation (NO gradients needed) ----
        y_pred = predict(
            weights, X, qml_circuit, encoding_fn, entanglement_fn, reuploading_count
        )
        acc = accuracy(Y, y_pred)

        if epoch % 10 == 0 or epoch == 99:
            print(
                f"Epoch {epoch + 1:3d} | "
                f"Cost = {current_cost:.4f} | "
                f"Accuracy = {acc:.3f}"
            )

        if epoch == 99:
            cm = confusion_matrix(Y, y_pred)
            print("\nConfusion Matrix (rows=true, cols=pred):")
            print(cm)



=== Reuploading count: 1 ===
Epoch   1 | Cost = 0.7054 | Accuracy = 0.500
Epoch  11 | Cost = 0.6949 | Accuracy = 0.500
Epoch  21 | Cost = 0.6932 | Accuracy = 0.500
Epoch  31 | Cost = 0.6935 | Accuracy = 0.500
Epoch  41 | Cost = 0.6932 | Accuracy = 0.500
Epoch  51 | Cost = 0.6932 | Accuracy = 0.500
Epoch  61 | Cost = 0.6932 | Accuracy = 0.500
Epoch  71 | Cost = 0.6931 | Accuracy = 0.500
Epoch  81 | Cost = 0.6931 | Accuracy = 0.500
Epoch  91 | Cost = 0.6931 | Accuracy = 0.500
Epoch 100 | Cost = 0.6931 | Accuracy = 0.500

Confusion Matrix (rows=true, cols=pred):
[[250   0]
 [250   0]]

=== Reuploading count: 3 ===
Epoch   1 | Cost = 1.0603 | Accuracy = 0.500
Epoch  11 | Cost = 0.8362 | Accuracy = 0.506
Epoch  21 | Cost = 0.7260 | Accuracy = 0.502
Epoch  31 | Cost = 0.7052 | Accuracy = 0.494
Epoch  41 | Cost = 0.6998 | Accuracy = 0.504
Epoch  51 | Cost = 0.6949 | Accuracy = 0.510
Epoch  61 | Cost = 0.6932 | Accuracy = 0.504
Epoch  71 | Cost = 0.6932 | Accuracy = 0.500
Epoch  81 | Cost = 0

## Krampus

In [20]:
opt = qml.AdamOptimizer(stepsize=0.01)

num_qubits = 2
dev = qml.device("default.qubit", wires=num_qubits)
qml_circuit = make_qml_circuit(dev)

encoding_fn = angle_encoding
entanglement_fn = circular_entanglement 

X = X_train[krampus]
Y = y_train[krampus]



for reuploading_count in (1, 3):
    print(f"\n=== Reuploading count: {reuploading_count} ===")

    weights = qnp.array(
        np.random.uniform(0, math.pi, size=(reuploading_count, num_qubits)),
        requires_grad=True
    )

    param = [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count]

    for epoch in range(100):
        args, current_cost = opt.step_and_cost(cost, weights, param)
        weights = args[0]

        # ---- evaluation (NO gradients needed) ----
        y_pred = predict(
            weights, X, qml_circuit, encoding_fn, entanglement_fn, reuploading_count
        )
        acc = accuracy(Y, y_pred)

        if epoch % 10 == 0 or epoch == 99:
            print(
                f"Epoch {epoch + 1:3d} | "
                f"Cost = {current_cost:.4f} | "
                f"Accuracy = {acc:.3f}"
            )

        if epoch == 99:
            cm = confusion_matrix(Y, y_pred)
            print("\nConfusion Matrix (rows=true, cols=pred):")
            print(cm)    


=== Reuploading count: 1 ===
Epoch   1 | Cost = 0.9885 | Accuracy = 0.500
Epoch  11 | Cost = 0.9575 | Accuracy = 0.502
Epoch  21 | Cost = 0.9379 | Accuracy = 0.500
Epoch  31 | Cost = 0.9248 | Accuracy = 0.502
Epoch  41 | Cost = 0.9199 | Accuracy = 0.496
Epoch  51 | Cost = 0.9052 | Accuracy = 0.502
Epoch  61 | Cost = 0.8965 | Accuracy = 0.498
Epoch  71 | Cost = 0.8925 | Accuracy = 0.500
Epoch  81 | Cost = 0.8908 | Accuracy = 0.498
Epoch  91 | Cost = 0.8901 | Accuracy = 0.498
Epoch 100 | Cost = 0.8899 | Accuracy = 0.500

Confusion Matrix (rows=true, cols=pred):
[[101 149]
 [101 149]]

=== Reuploading count: 3 ===
Epoch   1 | Cost = 0.8438 | Accuracy = 0.484
Epoch  11 | Cost = 0.7890 | Accuracy = 0.484
Epoch  21 | Cost = 0.7403 | Accuracy = 0.508
Epoch  31 | Cost = 0.7139 | Accuracy = 0.506
Epoch  41 | Cost = 0.7024 | Accuracy = 0.488
Epoch  51 | Cost = 0.6977 | Accuracy = 0.494
Epoch  61 | Cost = 0.6954 | Accuracy = 0.486
Epoch  71 | Cost = 0.6939 | Accuracy = 0.480
Epoch  81 | Cost = 0

## Lebkuchen

In [24]:
opt = qml.AdamOptimizer(stepsize=0.01)

num_qubits = 1
dev = qml.device("default.mixed", wires=num_qubits)
qml_circuit = make_qml_circuit(dev)

encoding_fn = density_matrix_encoding
entanglement_fn = full_entanglement 

X = X_train[lebkuchen]
Y = y_train[lebkuchen]


for reuploading_count in (1, 3):
    weights = qnp.array(
        np.random.uniform(0, math.pi, size=(reuploading_count, num_qubits)), 
        requires_grad=True
    )
    
    param = [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count]

    for epoch in range(100):
        args, current_cost = opt.step_and_cost(cost, weights, param)
        weights = args[0]
        if epoch % 10 == 0 or epoch == 99: 
            print(f"Epoch {epoch + 1}: Cost = {current_cost:.4f} Weights = {weights}")
    

Epoch 1: Cost = 1.2117 Weights = [[1.46325162]]
Epoch 11: Cost = 1.2081 Weights = [[1.46139378]]
Epoch 21: Cost = 1.2082 Weights = [[1.46012994]]
Epoch 31: Cost = 1.2081 Weights = [[1.46010363]]
Epoch 41: Cost = 1.2081 Weights = [[1.46040019]]
Epoch 51: Cost = 1.2080 Weights = [[1.46041285]]
Epoch 61: Cost = 1.2080 Weights = [[1.46017626]]
Epoch 71: Cost = 1.2080 Weights = [[1.46025275]]
Epoch 81: Cost = 1.2080 Weights = [[1.46029608]]
Epoch 91: Cost = 1.2080 Weights = [[1.46029657]]
Epoch 100: Cost = 1.2080 Weights = [[1.4603127]]
Epoch 1: Cost = 1.6090 Weights = [[0.24686091]
 [2.39591723]
 [0.92127121]]
Epoch 11: Cost = 1.2335 Weights = [[0.33117002]
 [2.48022634]
 [1.00558032]]
Epoch 21: Cost = 1.0651 Weights = [[0.41327445]
 [2.56233077]
 [1.08768475]]
Epoch 31: Cost = 0.9592 Weights = [[0.467422  ]
 [2.61647833]
 [1.1418323 ]]
Epoch 41: Cost = 0.8946 Weights = [[0.52596577]
 [2.67502209]
 [1.20037607]]
Epoch 51: Cost = 0.9006 Weights = [[0.53253494]
 [2.68159126]
 [1.20694524]]
E

In [25]:
def make_qml_circuit_2(dev):
    @qml.qnode(dev)
    def qml_circuit(weights, inputs, reuploading_count):
        # We repeat the (Encoding -> Entanglement -> Trainable) block
        for j in range(reuploading_count):
            
            # 1. Angle Encoding (Re-uploading)
            # We use RX to encode the first feature and RY for the second
            for i in range(num_qubits):
                qml.RX(inputs[0], wires=i)
                qml.RY(inputs[1], wires=i)
            
            # 2. Entanglement
            if num_qubits > 1:
                for i in range(num_qubits - 1):
                    qml.CNOT(wires=[i, i + 1])
            
            # 3. Trainable Layer (General Rotation)
            for i in range(num_qubits):
                # Using Rot (phi, theta, omega) for full Bloch sphere coverage
                qml.Rot(weights[j, i, 0], weights[j, i, 1], weights[j, i, 2], wires=i)

        return qml.expval(qml.PauliZ(0))
    
    return qml_circuit

In [27]:
# --- Setup ---
num_qubits = 2
reuploading_count = 3  # More layers = more complex decision boundaries
dev = qml.device("default.qubit", wires=num_qubits)
qml_circuit = make_qml_circuit_2(dev)

# --- Data ---
X = qnp.array(X_train[gluehwein], requires_grad=False)
Y = qnp.array(y_train[gluehwein], requires_grad=False)

# --- Initialization ---
# 3 weights per qubit per layer for qml.Rot
weights = qnp.array(
    np.random.uniform(0, 2 * np.pi, size=(reuploading_count, num_qubits, 3)),
    requires_grad=True
)

opt = qml.AdamOptimizer(stepsize=0.1) # Increased stepsize for faster convergence

# --- Training Loop ---
for epoch in range(100):
    # The cost function needs to be updated to match the new circuit signature
    def cost_fn(w):
        # Map expval [-1, 1] to [eps, 1-eps] for BCE loss
        predictions = qnp.array([
            (qml_circuit(w, x, reuploading_count) * 0.499 + 0.5)
            for x in X
        ])
        return -qnp.mean(Y * qnp.log(predictions) + (1 - Y) * qnp.log(1 - predictions))

    weights, current_cost = opt.step_and_cost(cost_fn, weights)

    if (epoch + 1) % 10 == 0:
        # Quick accuracy check
        y_pred = qnp.array([qml_circuit(weights, x, reuploading_count) for x in X]) > 0
        acc = qnp.mean(y_pred == Y)
        print(f"Epoch {epoch+1:3d} | Cost: {current_cost:.4f} | Acc: {acc:.3f}")

Epoch  10 | Cost: 0.4249 | Acc: 0.876
Epoch  20 | Cost: 0.3758 | Acc: 0.874
Epoch  30 | Cost: 0.2683 | Acc: 0.946
Epoch  40 | Cost: 0.2412 | Acc: 0.966
Epoch  50 | Cost: 0.2343 | Acc: 0.976
Epoch  60 | Cost: 0.2299 | Acc: 0.974
Epoch  70 | Cost: 0.2285 | Acc: 0.972
Epoch  80 | Cost: 0.2284 | Acc: 0.972
Epoch  90 | Cost: 0.2281 | Acc: 0.972
Epoch 100 | Cost: 0.2281 | Acc: 0.974
